In [9]:
import os
import logging
from datetime import datetime
from ipywidgets import Dropdown

In [2]:


import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\hemch\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [4]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [10]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import json
import logging
import sys
from pathlib import Path

import ipytest

#CODE_FOLDER = Path("code")
#sys.path.extend([f"./{CODE_FOLDER}"])

#DATA_FILEPATH = "../data/penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
import os

bucket = os.environ["BUCKET"]+'/sagemaker-course'
role = os.environ["ROLE"]

COMET_API_KEY = os.environ.get("COMET_API_KEY", None)
COMET_PROJECT_NAME = os.environ.get("COMET_PROJECT_NAME", None)

In [7]:
bucket

'beercafe-learn-sagemaker/sagemaker-course'

In [8]:
prefix = "iris"

# Download jumpstart manifest file

In [11]:
# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

print("number of models: ", len(model_list))

number of models:  4672


In [12]:
import pandas as pd
model_df = pd.DataFrame(model_list)
model_df.sample(10)

,model_id,version,min_version,spec_key,search_keywords
3185,tensorflow-ic-imagenet-mobilenet-v1-050-192-cl...,1.1.0,2.75.0,community_models/tensorflow-ic-imagenet-mobile...,NaN
756,huggingface-text2text-flan-t5-large,1.0.0,2.130.0,community_models/huggingface-text2text-flan-t5...,NaN
376,huggingface-sentencesimilarity-bge-small-en,1.0.0,2.144.0,community_models/huggingface-sentencesimilarit...,NaN
1092,huggingface-textgeneration1-gpt-neo-2-7b,3.0.0,2.189.0,community_models/huggingface-textgeneration1-g...,NaN
3860,tensorflow-icembedding-imagenet-mobilenet-v1-0...,2.0.2,2.80.0,community_models/tensorflow-icembedding-imagen...,NaN
292,huggingface-llm-mixtral-8x7b,1.1.0,2.189.0,community_models/huggingface-llm-mixtral-8x7b/...,NaN
3100,tensorflow-ic-imagenet-mobilenet-v1-025-160-cl...,3.0.2,2.80.0,community_models/tensorflow-ic-imagenet-mobile...,NaN
3124,tensorflow-ic-imagenet-mobilenet-v1-025-192-cl...,1.1.1,2.75.0,community_models/tensorflow-ic-imagenet-mobile...,NaN
811,huggingface-text2text-flan-t5-xxl-fp16,1.0.1,2.130.0,community_models/huggingface-text2text-flan-t5...,NaN
2680,tensorflow-ic-deit-tiny-distilled-patch16-224,1.0.2,2.80.0,community_models/tensorflow-ic-deit-tiny-disti...,NaN


# Filter Object detection models

In [13]:

# filter-out all the Object Detection models from the manifest list.
od_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-od-" in model_id or "-od1-" in model_id) and model_id not in od_models:
        od_models.append(model_id)

print(f"Number of od models available for inference: {len(od_models)}")

Number of od models available for inference: 70


In [14]:
# display the model-ids in a dropdown to select a model for inference.
infer_model_dropdown = Dropdown(
    options=od_models,
    value="pytorch-od-nvidia-ssd",
    description="Select a model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)

In [15]:
display(infer_model_dropdown)

Dropdown(description='Select a model:', index=17, layout=Layout(width='max-content'), options=('mxnet-od-faste…

In [18]:
print(infer_model_dropdown.value)

tensorflow-od-efficientdet-d0-1


# Filter Image classification models

In [19]:
# filter-out all the Image Classification models from the manifest list.
ic_models = []
for model in model_list:
    model_id = model["model_id"]
    if ("-ic-" in model_id) and model_id not in ic_models:
        ic_models.append(model_id)

print(f"Number of ic models available for inference: {len(ic_models)}")

Number of ic models available for inference: 162
